In [ ]:
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex SDK: Train and deploy an XGBoost model with pre-built containers (formerly hosted runtimes)


## Installation


Install the Google *cloud-storage* library as well.


In [ ]:
! pip3 install google-cloud-storage

### Restart the Kernel

Once you've installed the Vertex SDK and Google *cloud-storage*, you need to restart the notebook kernel so it can find the packages.


In [ ]:
import os

if not os.getenv("AUTORUN"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## Before you begin

### GPU run-time

*Make sure you're running this notebook in a GPU runtime if you have that option. In Colab, select* **Runtime > Change Runtime Type > GPU**

### Set up your GCP project

**The following steps are required, regardless of your notebook environment.**

1. [Select or create a GCP project](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 free credit towards your compute/storage costs.

2. [Make sure that billing is enabled for your project.](https://cloud.google.com/billing/docs/how-to/modify-project)

3. [Enable the Vertex APIs and Compute Engine APIs.](https://console.cloud.google.com/flows/enableapi?apiid=ml.googleapis.com,compute_component)

4. [Google Cloud SDK](https://cloud.google.com/sdk) is already installed in Google Cloud Notebooks.

5. Enter your project ID in the cell below. Then run the  cell to make sure the
Cloud SDK uses the right project for all the commands in this notebook.

**Note**: Jupyter runs lines prefixed with `!` as shell commands, and it interpolates Python variables prefixed with `$` into these commands.


In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = !gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

#### Region

You can also change the `REGION` variable, which is used for operations
throughout the rest of this notebook.  Below are regions supported for Vertex. We recommend when possible, to choose the region closest to you.

- Americas: `us-central1`
- Europe: `europe-west4`
- Asia Pacific: `asia-east1`

You cannot use a Multi-Regional Storage bucket for training with Vertex. Not all regions provide support for all Vertex services. For the latest support per region, see [Region support for Vertex services]()


In [ ]:
REGION = "us-central1"  # @param {type: "string"}

#### Timestamp

If you are in a live tutorial session, you might be using a shared test account or project. To avoid name collisions between users on resources created, you create a timestamp for each instance session, and append onto the name of resources which will be created in this tutorial.


In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### Authenticate your GCP account

**If you are using Google Cloud Notebooks**, your environment is already
authenticated. Skip this step.

*Note: If you are on an Vertex notebook and run the cell, the cell knows to skip executing the authentication steps.*


In [ ]:
import os
import sys

# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your Google Cloud account. This provides access
# to your Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

# If on Vertex, then don't execute this code
if not os.path.exists("/opt/deeplearning/metadata/env_version"):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this tutorial in a notebook locally, replace the string
    # below with the path to your service account key and run this cell to
    # authenticate your Google Cloud account.
    else:
        %env GOOGLE_APPLICATION_CREDENTIALS your_path_to_credentials.json

    # Log in to your account on Google Cloud
    ! gcloud auth login

### Create a Cloud Storage bucket

**The following steps are required, regardless of your notebook environment.**

This tutorial is designed to use training data that is in a public Cloud Storage bucket and a local Cloud Storage bucket for your batch predictions. You may alternatively use your own training data that you have stored in a local Cloud Storage bucket.

Set the name of your Cloud Storage bucket below. It must be unique across all Cloud Storage buckets.


In [ ]:
BUCKET_NAME = "[your-bucket-name]"  # @param {type:"string"}

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "[your-bucket-name]":
    BUCKET_NAME = PROJECT_ID + "aip-" + TIMESTAMP

**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.


In [ ]:
! gsutil mb -l $REGION gs://$BUCKET_NAME

Finally, validate access to your Cloud Storage bucket by examining its contents:


In [ ]:
! gsutil ls -al gs://$BUCKET_NAME

### Set up variables

Next, set up some variables used throughout the tutorial.
### Import libraries and define constants


#### Import Vertex SDK

Import the Vertex SDK into our Python environment.


In [ ]:
import json
import time

from googleapiclient import discovery

#### Vertex constants

Setup up the following constants for Vertex:

- `PARENT`: The Vertex location root path for dataset, model and endpoint resources.


In [ ]:
# Vertex location root path for your dataset, model and endpoint resources
PARENT = "projects/" + PROJECT_ID + "/locations/" + REGION

## Clients

The Vertex SDK works as a client/server model. On your side (the Python script) you will create a client that sends requests and receives responses from the server (Vertex).

You will use several clients in this tutorial, so set them all up upfront.


In [ ]:
client = discovery.build("ml", "v1")

## Prepare a trainer script

### Package assembly

In [ ]:
# Make folder for python training script
! rm -rf custom
! mkdir custom

# Add package information
! touch custom/README.md

setup_cfg = "[egg_info]\n\
tag_build =\n\
tag_date = 0"
! echo "$setup_cfg" > custom/setup.cfg

setup_py = "import setuptools\n\
setuptools.setup(\n\
    install_requires=[\n\
    ],\n\
    packages=setuptools.find_packages())"
! echo "$setup_py" > custom/setup.py

pkg_info = "Metadata-Version: 1.0\n\
Name: Custom XGBoost Iris\n\
Version: 0.0.0\n\
Summary: Demonstration training script\n\
Home-page: www.google.com\n\
Author: Google\n\
Author-email: aferlitsch@google.com\n\
License: Public\n\
Description: Demo\n\
Platform: Vertex AI"
! echo "$pkg_info" > custom/PKG-INFO

# Make the training subfolder
! mkdir custom/trainer
! touch custom/trainer/__init__.py

### Task.py contents

In [ ]:
%%writefile custom/trainer/task.py
# Single Instance Training for Iris

import datetime
import os
import subprocess
import sys
import pandas as pd
import xgboost as xgb

import argparse

parser = argparse.ArgumentParser()
parser.add_argument('--model-dir', dest='model_dir',
                    default=os.getenv('AIP_MODEL_DIR'), type=str, help='Model dir.')
args = parser.parse_args()

# Download data
iris_data_filename = 'iris_data.csv'
iris_target_filename = 'iris_target.csv'
data_dir = 'gs://cloud-samples-data/ai-platform/iris'

# gsutil outputs everything to stderr so we need to divert it to stdout.
subprocess.check_call(['gsutil', 'cp', os.path.join(data_dir,
                                                    iris_data_filename),
                       iris_data_filename], stderr=sys.stdout)
subprocess.check_call(['gsutil', 'cp', os.path.join(data_dir,
                                                    iris_target_filename),
                       iris_target_filename], stderr=sys.stdout)


# Load data into pandas, then use `.values` to get NumPy arrays
iris_data = pd.read_csv(iris_data_filename).values
iris_target = pd.read_csv(iris_target_filename).values

# Convert one-column 2D array into 1D array for use with XGBoost
iris_target = iris_target.reshape((iris_target.size,))


# Load data into DMatrix object
dtrain = xgb.DMatrix(iris_data, label=iris_target)

# Train XGBoost model
bst = xgb.train({}, dtrain, 20)

# Export the classifier to a file
model_filename = 'model.bst'
bst.save_model(model_filename)


# Upload the saved model file to Cloud Storage
gcs_model_path = os.path.join(args.model_dir, model_filename)
subprocess.check_call(['gsutil', 'cp', model_filename, gcs_model_path],
    stderr=sys.stdout)


### Store training script on your Cloud Storage bucket

In [ ]:
! rm -f custom.tar custom.tar.gz
! tar cvf custom.tar custom
! gzip custom.tar
! gsutil cp custom.tar.gz gs://$BUCKET_NAME/iris.tar.gz

## Train a model

### [projects.jobs.create](https://cloud.google.com/ai-platform/training/docs/reference/rest/v1/projects.jobs/create)

#### Request

In [ ]:
JOB_NAME = "custom_job_XGB" + TIMESTAMP

training_input = {
    "scaleTier": "BASIC",
    "packageUris": ["gs://" + BUCKET_NAME + "/iris.tar.gz"],
    "pythonModule": "trainer.task",
    "args": ["--model-dir=" + "gs://{}/{}".format(BUCKET_NAME, JOB_NAME)],
    "region": REGION,
    "runtimeVersion": "2.4",
    "pythonVersion": "3.7",
}

body = {"jobId": JOB_NAME, "trainingInput": training_input}

request = client.projects().jobs().create(parent="projects/" + PROJECT_ID)
request.body = body

print(json.dumps(json.loads(request.to_json()), indent=2))

request = client.projects().jobs().create(parent="projects/" + PROJECT_ID, body=body)

*Example output*:
```
{
  "uri": "https://ml.googleapis.com/v1/projects/migration-ucaip-training/jobs?alt=json",
  "method": "POST",
  "body": {
    "jobId": "custom_job_XGB20210302154841",
    "trainingInput": {
      "scaleTier": "BASIC",
      "packageUris": [
        "gs://migration-ucaip-trainingaip-20210302154841/iris.tar.gz"
      ],
      "pythonModule": "trainer.task",
      "args": [
        "--model-dir=gs://migration-ucaip-trainingaip-20210302154841/custom_job_XGB20210302154841"
      ],
      "region": "us-central1",
      "runtimeVersion": "2.4",
      "pythonVersion": "3.7"
    }
  },
  "headers": {
    "accept": "application/json",
    "accept-encoding": "gzip, deflate",
    "user-agent": "(gzip)",
    "x-goog-api-client": "gdcl/1.12.8 gl-python/3.7.8"
  },
  "methodId": "ml.projects.jobs.create",
  "resumable": null,
  "response_callbacks": [],
  "_in_error_state": false,
  "body_size": 0,
  "resumable_uri": null,
  "resumable_progress": 0
}
```


#### Call

In [ ]:
result = request.execute()

#### Response

In [ ]:
print(json.dumps(result, indent=2))

*Example output*:
```
{
  "jobId": "custom_job_XGB20210302154841",
  "trainingInput": {
    "packageUris": [
      "gs://migration-ucaip-trainingaip-20210302154841/iris.tar.gz"
    ],
    "pythonModule": "trainer.task",
    "args": [
      "--model-dir=gs://migration-ucaip-trainingaip-20210302154841/custom_job_XGB20210302154841"
    ],
    "region": "us-central1",
    "runtimeVersion": "2.4",
    "pythonVersion": "3.7"
  },
  "createTime": "2021-03-02T15:50:12Z",
  "state": "QUEUED",
  "trainingOutput": {},
  "etag": "PmcK2JEDnJM="
}
```


In [ ]:
# The short numeric ID for the custom training job
custom_training_short_id = result["jobId"]
# The full unique ID for the custom training job
custom_training_id = "projects/" + PROJECT_ID + "/jobs/" + result["jobId"]

print(custom_training_id)

### [projects.jobs.get](https://cloud.google.com/ai-platform/training/docs/reference/rest/v1/projects.jobs/get)

#### Call

In [ ]:
request = client.projects().jobs().get(name=custom_training_id)

result = request.execute()

#### Response

In [ ]:
print(json.dumps(result, indent=2))

*Example output*:
```
{
  "jobId": "custom_job_XGB20210302154841",
  "trainingInput": {
    "packageUris": [
      "gs://migration-ucaip-trainingaip-20210302154841/iris.tar.gz"
    ],
    "pythonModule": "trainer.task",
    "args": [
      "--model-dir=gs://migration-ucaip-trainingaip-20210302154841/custom_job_XGB20210302154841"
    ],
    "region": "us-central1",
    "runtimeVersion": "2.4",
    "pythonVersion": "3.7"
  },
  "createTime": "2021-03-02T15:50:12Z",
  "state": "PREPARING",
  "trainingOutput": {},
  "etag": "L+085Kgm1Wo="
}
```


In [ ]:
while True:
    response = client.projects().jobs().get(name=custom_training_id).execute()

    if response["state"] != "SUCCEEDED":
        print("Training job has not completed:", response["state"])
        if response["state"] == "FAILED":
            break
    else:
        break
    time.sleep(60)

# model artifact output directory on Google Cloud Storage
model_artifact_dir = response["trainingInput"]["args"][0].split("=")[-1]
print("artifact location  " + model_artifact_dir)

## Deploy the model

### [projects.models.create](https://cloud.google.com/ai-platform/prediction/docs/reference/rest/v1/projects.models/create)

#### Request

In [ ]:
body = {"name": "custom_job_XGB" + TIMESTAMP}

request = client.projects().models().create(parent="projects/" + PROJECT_ID)
request.body = json.loads(json.dumps(body, indent=2))

print(json.dumps(json.loads(request.to_json()), indent=2))

request = client.projects().models().create(parent="projects/" + PROJECT_ID, body=body)

*Example output*:
```
{
  "uri": "https://ml.googleapis.com/v1/projects/migration-ucaip-training/models?alt=json",
  "method": "POST",
  "body": {
    "name": "custom_job_XGB20210302154841"
  },
  "headers": {
    "accept": "application/json",
    "accept-encoding": "gzip, deflate",
    "user-agent": "(gzip)",
    "x-goog-api-client": "gdcl/1.12.8 gl-python/3.7.8"
  },
  "methodId": "ml.projects.models.create",
  "resumable": null,
  "response_callbacks": [],
  "_in_error_state": false,
  "body_size": 0,
  "resumable_uri": null,
  "resumable_progress": 0
}
```


#### Call

In [ ]:
result = request.execute()

#### Response

In [ ]:
print(json.dumps(result, indent=2))

*Example output*:
```
{
  "name": "projects/migration-ucaip-training/models/custom_job_XGB20210302154841",
  "regions": [
    "us-central1"
  ],
  "etag": "4gQZjQgH2sc="
}
```


In [ ]:
model_id = result["name"]

### [projects.models.versions.create](https://cloud.google.com/ai-platform/prediction/docs/reference/rest/v1/projects.models.versions/create)

#### Request

In [ ]:
version = {
    "name": "custom_job_XGB" + TIMESTAMP,
    "deploymentUri": model_artifact_dir,
    "runtimeVersion": "2.1",
    "framework": "XGBOOST",
    "pythonVersion": "3.7",
    "machineType": "mls1-c1-m2",
}

request = (
    client.projects()
    .models()
    .versions()
    .create(
        parent=model_id,
    )
)
request.body = version

print(json.dumps(json.loads(request.to_json()), indent=2))

request = client.projects().models().versions().create(parent=model_id, body=version)

*Example output*:
```
{
  "uri": "https://ml.googleapis.com/v1/projects/migration-ucaip-training/models/custom_job_XGB20210302154841/versions?alt=json",
  "method": "POST",
  "body": {
    "name": "custom_job_XGB20210302154841",
    "deploymentUri": "gs://migration-ucaip-trainingaip-20210302154841/custom_job_XGB20210302154841",
    "runtimeVersion": "2.1",
    "framework": "XGBOOST",
    "pythonVersion": "3.7",
    "machineType": "mls1-c1-m2"
  },
  "headers": {
    "accept": "application/json",
    "accept-encoding": "gzip, deflate",
    "user-agent": "(gzip)",
    "x-goog-api-client": "gdcl/1.12.8 gl-python/3.7.8"
  },
  "methodId": "ml.projects.models.versions.create",
  "resumable": null,
  "response_callbacks": [],
  "_in_error_state": false,
  "body_size": 0,
  "resumable_uri": null,
  "resumable_progress": 0
}
```


#### Call

In [ ]:
result = request.execute()

#### Response

In [ ]:
print(json.dumps(result, indent=2))

*Example output*:
```
{
  "name": "projects/migration-ucaip-training/operations/create_custom_job_XGB20210302154841_custom_job_XGB20210302154841-1614701495149",
  "metadata": {
    "@type": "type.googleapis.com/google.cloud.ml.v1.OperationMetadata",
    "createTime": "2021-03-02T16:11:35Z",
    "operationType": "CREATE_VERSION",
    "modelName": "projects/migration-ucaip-training/models/custom_job_XGB20210302154841",
    "version": {
      "name": "projects/migration-ucaip-training/models/custom_job_XGB20210302154841/versions/custom_job_XGB20210302154841",
      "deploymentUri": "gs://migration-ucaip-trainingaip-20210302154841/custom_job_XGB20210302154841",
      "createTime": "2021-03-02T16:11:35Z",
      "runtimeVersion": "2.1",
      "etag": "t71tF0fa60o=",
      "framework": "XGBOOST",
      "machineType": "mls1-c1-m2",
      "pythonVersion": "3.7"
    }
  }
}
```


In [ ]:
# The full unique ID for the model version
model_version_name = result["metadata"]["version"]["name"]

print(model_version_name)

In [ ]:
while True:
    response = (
        client.projects().models().versions().get(name=model_version_name).execute()
    )
    if response["state"] == "READY":
        print("Model version created.")
        break
    time.sleep(60)

## Make batch predictions


Batch prediction only supports Tensorflow. FRAMEWORK_SCIKIT_LEARN is not currently available.

## Make online predictions

In [ ]:
INSTANCES = [[1.4, 1.3, 5.1, 2.8], [1.5, 1.2, 4.7, 2.4]]

### Prepare file for online prediction

### [projects.predict](https://cloud.google.com/ai-platform/prediction/docs/reference/rest/v1/projects/predict)

#### Request

In [ ]:
request = client.projects().predict(
    name=model_version_name,
)
request.body = json.loads(json.dumps({"instances": INSTANCES}, indent=2))

print(json.dumps(json.loads(request.to_json()), indent=2))

request = client.projects().predict(
    name=model_version_name, body={"instances": INSTANCES}
)

*Example output*:
```
{
  "uri": "https://ml.googleapis.com/v1/projects/migration-ucaip-training/models/custom_job_XGB20210302154841/versions/custom_job_XGB20210302154841:predict?alt=json",
  "method": "POST",
  "body": {
    "instances": [
      [
        1.4,
        1.3,
        5.1,
        2.8
      ],
      [
        1.5,
        1.2,
        4.7,
        2.4
      ]
    ]
  },
  "headers": {
    "accept": "application/json",
    "accept-encoding": "gzip, deflate",
    "user-agent": "(gzip)",
    "x-goog-api-client": "gdcl/1.12.8 gl-python/3.7.8"
  },
  "methodId": "ml.projects.predict",
  "resumable": null,
  "response_callbacks": [],
  "_in_error_state": false,
  "body_size": 0,
  "resumable_uri": null,
  "resumable_progress": 0
}
```


#### Call

In [ ]:
result = request.execute()

#### Response

In [ ]:
print(json.dumps(result, indent=2))

*Example output*:
```
{
  "predictions": [
    [],
    []
  ]
}
```


### [projects.models.versions.delete](https://cloud.google.com/ai-platform/prediction/docs/reference/rest/v1/projects.models.versions/delete)

#### Request

In [ ]:
request = client.projects().models().versions().delete(name=model_version_name)

#### Call

In [ ]:
response = request.execute()

#### Response


In [ ]:
print(json.dumps(response, indent=2))

*Example output*:
```
{
  "name": "projects/migration-ucaip-training/operations/delete_custom_job_XGB20210302154841_custom_job_XGB20210302154841-1614709380234",
  "metadata": {
    "@type": "type.googleapis.com/google.cloud.ml.v1.OperationMetadata",
    "createTime": "2021-03-02T18:23:00Z",
    "operationType": "DELETE_VERSION",
    "modelName": "projects/migration-ucaip-training/models/custom_job_XGB20210302154841",
    "version": {
      "name": "projects/migration-ucaip-training/models/custom_job_XGB20210302154841/versions/custom_job_XGB20210302154841",
      "deploymentUri": "gs://migration-ucaip-trainingaip-20210302154841/custom_job_XGB20210302154841",
      "createTime": "2021-03-02T16:11:35Z",
      "runtimeVersion": "2.1",
      "state": "READY",
      "etag": "sBx1RZUe3HQ=",
      "framework": "XGBOOST",
      "machineType": "mls1-c1-m2",
      "pythonVersion": "3.7"
    }
  }
}
```


# Cleaning up

To clean up all GCP resources used in this project, you can [delete the GCP
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial.


In [ ]:
delete_model = True
delete_bucket = True

# Delete the model using the Vertex fully qualified identifier for the model
try:
    if delete_model:
        client.projects().models().delete(name=model_id)
except Exception as e:
    print(e)

if delete_bucket and "BUCKET_NAME" in globals():
    ! gsutil rm -r gs://$BUCKET_NAME